In [ ]:
import ROOT as R
import numpy as np
import psutil
import os
import gc

pid = os.getpid()
py = psutil.Process(pid)

print(py.memory_info())

Testing reshape to find GetIndex function:

In [ ]:
a = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
print(a,'\n')
print(a[0],'\n')
print(a[0,0],'\n')
print(a[0,0,0])

In [ ]:
nx = 3
ny = 4
nz = 5
b = np.arange(nx*ny*nz)
print(b)
c = b.reshape(nx,ny,nz)
print(c)
# print(c[0,0,0])#z,y,x
print(c[1,0,0])#z
print(c[0,1,0])#y
print(c[0,0,1])#x

In [ ]:
def find_index(ix,iy,iz,nx,ny,nz):
    part_tau_direction = nz*iy
    part_pf_direction  = iz
    part_fe_direction  = nz*ny*ix
    return part_fe_direction + part_pf_direction + part_tau_direction

In [ ]:
ix,iy,iz = 2,3,4
print(c[ix,iy,iz])
print(b[find_index(ix,iy,iz,nx,ny,nz)])

## Read data with header:

In [ ]:
import ROOT as R
import numpy as np

If the kernel dies by running the next line, run this line in a separate .py in the terminal. This will show you the errors:

In [ ]:
R.gInterpreter.ProcessLine('#include "DataLoader.h"')

In [ ]:
n_tau = 10
n_pf  = 5
n_fe  = 30
n_counts = 2

In [ ]:
data_loader = R.DataLoader('/data/store/reco_skim_v1/tau_DYJetsToLL_M-50.root', n_tau)  

In [ ]:
data = data_loader.LoadNext()
n_batches = data_loader.NumberOfBatches()

In [ ]:
def generator():
    while True:
        data_loader.Reset()
        while data_loader.HasNext():
            data = data_loader.LoadNext()
            x_np = np.asarray(data.x)
            x_3d = x_np.reshape((n_tau, n_pf, n_fe))
            y_np = np.asarray(data.y)
            y_2d = y_np.reshape((n_tau, n_counts))
            yield x_3d, y_2d

### Some tests:

In [ ]:
x_np = np.asarray(data.x)
print(x_np)

In [ ]:
n_fe  = 3
n_tau = 4
n_pf  = 2
x_3d = x_np.reshape((n_tau, n_pf, n_fe))
print(x_3d)

In [ ]:
x_3d[1, 1, :]

In [ ]:
d = 1
f = 9
print(type((d,f)))

In [ ]:
(x,y) = next(generator())

In [ ]:
print((x,y),'\n')
print((x[0,0,:],y[0,:]))

In [ ]:
print(y)

In [ ]:
y = np.array([[1,0],[0., 0.],[0., 1.],[0., 0.],[0., 1.],[0., 0.],[0., 1.],[0., 0.],[0., 1.],[0., 0.]])

## Build model:

In [ ]:
import tensorflow as tf

In [ ]:
inputs = tf.keras.Input(shape=(n_pf,n_fe))

In [ ]:
x_inputs = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)

In [ ]:
outputs = tf.keras.layers.Dense(2, activation=tf.nn.relu)(x_inputs)

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

fit(
    x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None,
    validation_split=0.0, validation_data=None, shuffle=True, class_weight=None,
    sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_batch_size=None, validation_freq=1,
    max_queue_size=10, workers=1, use_multiprocessing=False
)

In [ ]:
model.fit(x = tf.data.Dataset.from_generator(generator,(tf.float32, tf.float32),(tf.TensorShape([None,5,30]), tf.TensorShape([None,2]))), epochs = 1, steps_per_epoch = n_batches)

In [ ]:
tf.__version__

### Implement model as a class

In [ ]:
import tensorflow as tf

In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(2, activation=tf.nn.relu)

  def call(self, inputs):
    x_inputs = self.dense1(inputs2)
    return self.dense2(x_inputs)

model2 = MyModel()

In [ ]:
model2.compile(optimizer="Adam", loss="mse", metrics=["mae"])

In [ ]:
model2.fit(x = generator,epochs = 1, steps_per_epoch = n_batches)